In [ ]:
from DrissionPage import WebPage
import json

In [ ]:
driver = WebPage()

In [ ]:
# Start listening for the API request
driver.listen.start("https://www.crunchbase.com/v4/data/lists/organization.companies/414e9c05-7475-48d9-9082-9c0b293cfb79")

In [ ]:
# Navigate to the page to trigger the request
driver.get("https://www.crunchbase.com/discover/saved/companies/414e9c05-7475-48d9-9082-9c0b293cfb79")

In [ ]:
# Wait for and capture the request
packet = driver.listen.wait()
print("Request captured!")
print(f"URL: {packet.url}")
print(f"Method: {packet.method}")
print(f"Status: {packet.response.status}")

In [ ]:
# Extract headers and cookies
headers = dict(packet.request.headers)
cookies = driver.cookies()

print("\nRequest Headers:")
for key, value in headers.items():
    print(f"{key}: {value}")

print("\nCookies:")
for cookie in cookies:
    print(f"{cookie['name']}: {cookie['value'][:50]}..." if len(cookie['value']) > 50 else f"{cookie['name']}: {cookie['value']}")

In [ ]:
# Get the request body/payload
request_body = packet.request.postData
print("\nRequest Payload:")
payload = json.loads(request_body)
print(json.dumps(payload, indent=2))

In [ ]:
# Get the response data
response_data = packet.response.body
print("\nResponse Data:")
data = json.loads(response_data)
print(f"Count: {data.get('count', 0)} companies")
print(f"\nFirst 500 chars:")
print(json.dumps(data, indent=2)[:500])

In [ ]:
# Display companies
if data and data.get('entities'):
    print(f"\n✓ Found {data.get('count', 0)} companies!\n")
    print("="*80)
    for i, entity in enumerate(data['entities'], 1):
        props = entity.get('properties', {})
        name = props.get('identifier', {}).get('value', 'N/A')
        desc = props.get('short_description', {}).get('value', 'N/A')
        
        print(f"\n{i}. {name}")
        if desc and desc != 'N/A':
            print(f"   {desc[:150]}{'...' if len(desc) > 150 else ''}")
        print("-"*80)

In [ ]:
# Save all captured data
captured_data = {
    "url": packet.url,
    "method": packet.method,
    "headers": headers,
    "payload": payload,
    "response": data
}

with open('network_capture.json', 'w') as f:
    json.dump(captured_data, f, indent=2)
    
print("\nAll data saved to network_capture.json")